<a href="https://colab.research.google.com/github/yy3462-create/week1/blob/main/%E2%80%9Csession4_streamlit_colab_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Session 4 — Streamlit Mini Dashboard (Colab, no API keys)
This notebook launches a Streamlit app from Colab and exposes it via a **public URL**

You’ll build:
- CSV upload **or** sample dataset
- Sidebar filters + KPIs
- Plotly bar & scatter charts
- Download filtered CSV

## What is Streamlit and why use it for policy analysis?  

Streamlit is a Python library that lets you turn data scripts into interactive web apps with only a few extra lines of code—no need to learn web development. For public policy students, this is powerful because it allows you to move from static reports and spreadsheets to dynamic dashboards that let policymakers, colleagues, or the public **explore the data themselves**. Instead of handing someone a 30-page PDF, you can create a simple tool where users adjust filters, see charts update instantly, and even download customized slices of the data. This skill makes your analysis more transparent, accessible, and persuasive—key strengths in today’s policy environment where decisions often depend on clear, interactive communication of evidence.

## How the public/temporary link works (in Colab)

When you run Streamlit in Colab, the app runs on the Colab VM (not the public web).  
To let you (and others) open it in a browser, we start a **temporary tunnel**:

- **LocalTunnel / Cloudflare “trycloudflare”** creates a *public URL* that forwards traffic to your Colab session.
- The link is **ephemeral**: it changes each time you run the cell and stops working when the Colab runtime sleeps, restarts, or you stop the cell.
- Sometimes there’s a **password** (LocalTunnel) for basic protection. Anyone with the URL (and password, if any) can access the app while the cell is running.

This is perfect for **in-class demos, quick sharing, and short exercises**, but it’s not a durable deployment.

---

## What it takes to make a more permanent dashboard

To keep a Streamlit app available 24/7 at a stable URL, deploy it to a hosting service. Common paths:

### 1) Streamlit Community Cloud (easiest)
- **What you need**: A public GitHub repo with `app.py` and `requirements.txt`.
- **Steps**: Sign in at streamlit.io → “New app” → pick your repo/branch/file → Deploy.
- **Pros**: Free tier, zero-ops, simple secrets management.
- **Cons**: Public repos only (on free tier), resource limits.

### 2) Hugging Face Spaces
- **What you need**: Repo with `app.py`, `requirements.txt`, and `runtime.txt` (optional).
- **Pros**: Free starter tier, easy, nice for demos.
- **Cons**: Public by default on free; compute limits.

### 3) General-purpose hosts (more control)
- **Examples**: Render, Railway, Fly.io, Google Cloud Run, Azure App Service, AWS App Runner/ECS, Heroku (paid).
- **What you need**:
  - `app.py` (your Streamlit code)
  - `requirements.txt` (Python deps)
  - A startup command like `streamlit run app.py --server.port $PORT --server.headless true`
  - (Some platforms) a `Procfile` or Dockerfile
- **Pros**: Private repos, custom domains, more CPU/RAM.
- **Cons**: More setup/maintenance, may incur costs.

---

## Data & security considerations (for policy projects)

- **Secrets**: Store API keys in platform-provided secrets (Streamlit Cloud “Secrets”, HF Spaces “Secrets”, Render/Cloud env vars). **Never** hardcode keys in the repo.
- **Data location**:
  - **Static**: Put a CSV in the repo for reproducible demos.
  - **Dynamic**: Pull from a database (Postgres), a data warehouse, or Google Sheets/API.
- **Privacy**: If data includes PII/sensitive fields, sanitize or restrict access (see below).
- **Auth**: For private dashboards, add a login layer:
  - `streamlit-authenticator` (simple usernames/passwords)
  - Reverse proxy (NGINX/Cloud) with basic auth or SSO
  - Host on a private network/VPC


## 0) Install dependencies (Streamlit, Plotly, plus tunneling via LocalTunnel)

In [ ]:

!pip -q install streamlit pandas plotly python-dotenv
# Install localtunnel for a public URL without accounts
!npm -g -s install localtunnel



## 1) Create `app.py`
- CSV upload or sample dataset (penguins)
- Sidebar filters (up to 2 categorical columns)
- KPIs
- Plotly bar + scatter
- Notes box and CSV download


In [ ]:
%%writefile app.py
# 👆 This magic command tells Colab to save everything below into a file named app.py

# --- Imports ---
import io                         # lets us handle file uploads in memory
import pandas as pd               # data handling (tables, filters, summaries)
import plotly.express as px       # easy charts
import streamlit as st            # makes interactive dashboards

# --- Streamlit setup ---
st.set_page_config(page_title="Policy Mini Dashboard", layout="wide")
st.title("📊 Policy Mini Dashboard — Streamlit (Colab)")
st.caption("Upload a CSV or use the sample dataset; filter, chart, and download a cleaned slice.")

# --- Data source (sample CSV for students without their own) ---
SAMPLE_URL = "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv"

# --- Sidebar: Data choice ---
st.sidebar.header("Data")
use_sample = st.sidebar.toggle("Use sample dataset (penguins)", value=True)   # switch for using sample
uploaded = st.sidebar.file_uploader("...or upload a CSV", type=["csv"])      # upload your own CSV

# --- Cache loading so we don’t reload file each time a widget changes ---
@st.cache_data(show_spinner=False)
def load_csv(src: str | io.BytesIO) -> pd.DataFrame:
    return pd.read_csv(src)

# Decide which dataset to use
if use_sample:
    df = load_csv(SAMPLE_URL)
else:
    if uploaded is not None:
        df = load_csv(uploaded)
    else:
        st.info("Upload a CSV or toggle the sample dataset.")
        st.stop()   # stop here if no data is available

# --- Basic info + preview ---
st.success(f"Loaded **{len(df):,} rows × {len(df.columns)} columns**")   # show size of dataset
with st.expander("Preview data"):
    st.dataframe(df.head(20), use_container_width=True)                  # show first 20 rows

# --- Sidebar filters (limit to 2 categorical filters to keep simple) ---
st.sidebar.header("Filters")
cat_cols = [c for c in df.columns if df[c].dtype == "object"]            # categorical columns
num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]  # numeric columns

filters = {}
for c in cat_cols[:2]:
    vals = sorted([v for v in df[c].dropna().unique()])
    default_vals = vals[:2] if len(vals) > 2 else vals
    chosen = st.sidebar.multiselect(f"{c} filter", vals, default=default_vals)
    if chosen:
        filters[c] = chosen

# Apply filters
fdf = df.copy()
for c, vals in filters.items():
    fdf = fdf[fdf[c].isin(vals)]

# --- Guard against empty dataset after filtering ---
if len(fdf) == 0:
    st.warning("No rows match your current filters. Try removing or changing a filter.")
    st.stop()

# --- KPIs (key numbers) ---
col1, col2, col3 = st.columns(3)
with col1:
    st.metric("Rows after filter", f"{len(fdf):,}")
with col2:
    st.metric("Numeric columns", f"{len(num_cols)}")
with col3:
    st.metric("Missing values", int(fdf.isna().sum().sum()))

# --- Charts ---
st.markdown("### Charts")

# Bar chart (counts by first categorical column)
if cat_cols:
    c = cat_cols[0]
    count_df = fdf[c].fillna("Missing").value_counts(dropna=False).reset_index()
    count_df.columns = [c, "count"]
    fig_bar = px.bar(
        count_df,
        x=c, y="count",
        title=f"Count by {c}",
        category_orders={c: count_df.sort_values('count', ascending=False)[c].tolist()}
    )
    st.plotly_chart(fig_bar, use_container_width=True)

# Scatter plot (pick 2 numeric columns)
if len(num_cols) >= 2:
    xcol = st.selectbox("X-axis (numeric)", num_cols, index=0)
    ycol = st.selectbox("Y-axis (numeric)", num_cols, index=1)
    color = st.selectbox("Color by (optional)", ["(none)"] + cat_cols, index=0)

    if pd.api.types.is_numeric_dtype(fdf[xcol]) and pd.api.types.is_numeric_dtype(fdf[ycol]):
        fig_scatter = px.scatter(
            fdf, x=xcol, y=ycol,
            color=None if color == "(none)" else color,
            title=f"{xcol} vs {ycol}"
        )
        st.plotly_chart(fig_scatter, use_container_width=True)
    else:
        st.info("Pick two numeric columns for a scatter plot.")

# --- Notes & Insights (students can type observations here) ---
st.markdown("### 📝 Notes & Insights (manual)")
text_key = "notes_text"
st.session_state.setdefault(text_key, "")
st.session_state[text_key] = st.text_area(
    "Write a brief interpretation of what you see (for your README or summary):",
    value=st.session_state[text_key],
    height=120
)
st.caption("Tip: Use the AI Prompt Pack in this notebook to draft plain-English insights (in your browser).")

# --- Download filtered data ---
st.download_button("Download filtered CSV", fdf.to_csv(index=False), file_name="filtered.csv")
st.caption("Built in Streamlit • Change filters and chart columns to explore.")


Overwriting app.py



## 2) Run Streamlit with a **LocalTunnel** public URL (no auth required)

- Runs Streamlit in the background
- Exposes port **8501** via LocalTunnel
- Prints a clickable public link

> If the link times out, stop the cell and re-run it.


In [ ]:
# Install cloudflared (one-time per runtime)
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb >/dev/null 2>&1 || true

# Run Streamlit in the background and expose it via Cloudflare Tunnel
import subprocess, time, re, sys

PORT = "8501"
streamlit = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", PORT, "--server.headless", "true"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
time.sleep(2)

tunnel = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", f"http://localhost:{PORT}", "--no-autoupdate"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)

print("Starting Cloudflare Tunnel… (keep this cell running)")
for line in tunnel.stdout:
    sys.stdout.write(line)
    sys.stdout.flush()
    # The public URL looks like: https://<random>.trycloudflare.com
    m = re.search(r"https://[a-z0-9-]+\.trycloudflare\.com", line)
    if m:
        print("\n🚀 Streamlit app URL:", m.group(0))
        print("No password required. Keep this cell running while you use the app.")
        break


Starting Cloudflare Tunnel… (keep this cell running)
2025-09-30T23:56:42Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-09-30T23:56:42Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-09-30T23:56:45Z INF +--------------------------------------------------------------------------------------------+
2025-09-30T23:56:45Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-09-30T23:56:45Z INF |  


---

# 📌 AI Prompt Pack (use in ChatGPT or Gemini — browser only)

**How to:** Copy a prompt, paste it in the AI chat, add context (tiny CSV sample, code, or error), and ask for the **smallest** change or a **single** code block. Never paste secrets.

### Minimal Streamlit app from my CSV
```
You're my coding mentor. I’m in Colab and want a minimal Streamlit app:
- Load a CSV (or penguins sample URL)
- Preview + sidebar multiselect filters (2 categorical)
- One Plotly bar + one scatter (two numeric, optional color)
Return a single `app.py`. Assume this CSV head reflects my schema:

[PASTE df.head(20).to_csv(index=False) HERE]

My goal: [short plain-English chart goal]
```

### Add “Notes & Insights” box (manual)
```
Add to my Streamlit app a text area “Notes & Insights” using session_state, and a Download button for filtered CSV. Show only the additional code and where to insert.
```

### Debug: Fix this error
```
Help me fix this Streamlit/Plotly error:
1) Root cause in plain English
2) Smallest code change
3) Corrected snippet

Traceback:
[PASTE ERROR HERE]
```

### Empty chart guard
```
My filtered dataframe is empty. Give a quick checklist and a guard clause that warns and stops when len(fdf)==0.
```

### Summarize to a policymaker
```
Give 3 bullet insights (1 sentence each), 1 caveat, and 1 follow-up question (≤120 words) using this sample:
[PASTE CSV SAMPLE HERE]
```

### Suggest 3 charts
```
Given these columns & types:
[PASTE COLUMNS]
Suggest 3 Plotly Express charts (x, y, color) with 1-sentence “why this matters” each.
```

### Reusable Plotly helpers
```
Write `make_bar(df, cat_col, title)` that handles NaNs as "Missing",
sorts descending, rotates labels if >8 categories, adds text when counts<50.
Return only the function.
```
```
Create `make_scatter(df, x, y, color=None)` with linear trendline,
simple hovertemplate, and guards for non-numeric x/y. Return function + tiny usage example.
```

### UX copy
```
Write friendly 60-word sidebar instructions for students: upload or sample, apply filters, what KPIs/charts show.
```
```
Write 3 encouraging empty-state messages when filters remove all rows.
```

### Ethics & limits
```
≤80 words for an expander “Data cautions”: sampling/coverage bias, missing values, ecological fallacy, read the codebook.
```
```
Two-sentence warning under scatter: correlation ≠ causation; confounders may exist.
```
